This is a quick example showing how to use the code with the ISBI 2012 data set.

Note that the sample weights provided here are *not* the best possible - for best results one would want to train for longer.

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import os, sys, time
from socket import gethostname

import numpy as np
import pylab as plt
import matplotlib.gridspec as gridspec
import SimpleITK as sitk #you will need to install the simpleitk package on your computer

np.random.seed(9999)

import keras #keras and tensorflow need to be at the most current version, or it might not work
from keras import backend as K
from keras.models import load_model

sys.path.append('../..')
from cnn_tools import *
from data_tools import *


K.set_image_dim_ordering('th')

print('Using Keras version: %s' % keras.__version__)

In [ ]:
# load data
isbi_dir = os.path.expanduser('~/CIRCUIT/team1/completed/1605') #this is the path to the folder with your niftii data

test_nii = sitk.ReadImage(os.path.join(isbi_dir, 'team1_waypoint_201610_full_May6_25k_201610_dataset_em_1695.nii.gz'))
test_nii = sitk.Cast(test_nii, sitk.sitkUInt8)
X_test = sitk.GetArrayFromImage(test_nii) #converts image to array
X_test = X_test / 255. #scales array values
X_test = np.expand_dims(X_test, axis=1) #adds the color channel dimension to array

train_nii = sitk.ReadImage(os.path.join(isbi_dir, 'team1_waypoint_201610_full_May6_25k_201610_dataset_em_1640.nii.gz'))
train_nii = sitk.Cast(train_nii, sitk.sitkUInt8)
X_train = sitk.GetArrayFromImage(train_nii)
X_train = X_train / 255.
X_train = np.expand_dims(X_train, axis=1)


label_nii = sitk.ReadImage(os.path.join(isbi_dir, 'team1_waypoint_201610_full_May6_25k_201610_dataset_em_1640_mem.nii.gz'))
label_nii = sitk.Cast(label_nii, sitk.sitkUInt8)
Y_train = sitk.GetArrayFromImage(label_nii)
Y_train[Y_train==255] = 1
Y_train = np.expand_dims(Y_train, axis=1)


print('[info]: test data has shape:     ', X_test.shape)
print('[info]: train data has shape:     ', X_train.shape)

# load model
model = create_unet((1, 256, 256))
model.load_weights('./U-Net_weights_epoch0003.hdf5')

print('[info]: model input shape: ', model.input_shape)

In [ ]:
# deploy on test data
tic = time.time()

Y_hat_test = deploy_model(X_test, model, two_pass=True)

print('[info]: total time to deploy on system "%s": %0.2f sec' % (gethostname(), (time.time() - tic)))
print('[info]: Y_hat shape: ', Y_hat_test.shape)

In [ ]:
# visualize performance on test data
# Note: ISBI does not make truth available for test data.

for z in [0, 1, 2, 3, 4]:
    plt.figure(figsize=(8,8))
    plt.subplot(1,3,1)
    plt.imshow(X_test[z,0,...], cmap='bone'); plt.axis('off')
    plt.title('x[%d]' % z)
    plt.subplot(1,3,2)
    plt.imshow(Y_hat_test[z,0,...], cmap='magma'); plt.axis('off')
    plt.title('class 0 est')
    plt.subplot(1,3,3)
    plt.imshow(Y_hat_test[z,1,...], cmap='magma'); plt.axis('off')
    plt.title('class 1 est')
    plt.show()
    
# make sure the classification layers are as expected
net0 = Y_hat_test[0,0,...] + Y_hat_test[0,1,...]

plt.figure()
plt.hist(net0.flatten())
plt.xlim((.98, 1.02))

print(np.mean(net0))
print(np.std(net0))


In [ ]:
# deploy on train data
tic = time.time()
Y_hat_train = deploy_model(X_train, model)
Y_hat_train_2 = deploy_model(X_train, model, two_pass=True)
print('[info]: total time to deploy on system "%s": %0.2f sec' % (gethostname(), (time.time() - tic)))
print('[info]: Y_hat shape: ', Y_hat_train.shape)

In [ ]:
# also take a quick look at some training data

for z in [0,1,2,3,4]:
    plt.figure(figsize=(40,16))
    
    plt.subplot(2,4,1)
    plt.imshow(X_train[z,0,...], cmap='bone'); plt.axis('off')
    
    plt.subplot(2,4,2)
    plt.imshow(1-Y_train[z,0,...], cmap='magma'); plt.axis('off')
    plt.title('y_train[%d]' % z)
    
    plt.subplot(2,4,3)
    plt.imshow(Y_hat_train[z,1,...], cmap='magma'); plt.axis('off')
    plt.title('y_hat[%d]' % z)
    
    plt.subplot(2,4,4)
    plt.imshow(np.abs(Y_hat_train[z,1,...] - (1-Y_train[z,0,...])), cmap='jet'); plt.axis('off')
    plt.title('err[%d]' % z)
    
    plt.subplot(2,4,7)
    plt.imshow(Y_hat_train_2[z,1,...], cmap='magma'); plt.axis('off')
    
    plt.subplot(2,4,8)
    plt.imshow(np.abs(Y_hat_train_2[z,1,...] - (1-Y_train[z,0,...])), cmap='jet'); plt.axis('off')
    plt.title('err2')
    
    plt.subplots_adjust(wspace=0.05, hspace=.05)
    plt.show();